### Visualise the result of the station mapping
1. Import and define mapping output data layers to visualise

In [1]:
# Import necessary components from your module
from ipyleaflet import WidgetControl
from hat.mapping.visualisation import (
    GeoJSONLayerManager,
    InteractiveMap,
    create_gradient_legend,
    make_line_click_handler,
    make_style_callback,
    vector_style,
    create_polygon_legend,
)
from matplotlib import colormaps as cm, pyplot as plt


# Define a colormap and normalization for legend of optimum grid area diff (if needed)
cmap = cm["PRGn"]
vmin, vmax = -10, 10
norm = plt.Normalize(vmin=vmin, vmax=vmax)


# add various geojson layers
station_layer = GeoJSONLayerManager(
    "stations.geojson",  # geojson file
    style_callback=lambda feature: vector_style(feature, "blue", 0.5), # style
    name="Station", # name for legend
)
nearest_grid_layer = GeoJSONLayerManager(
    "near_grid.geojson", # geojson file
    style_callback=lambda feature: vector_style(feature, "black", 0.5), # style
    name="Nearest grid", # name for legend
)
optimum_grid_layer = GeoJSONLayerManager(
    "optimum_grid.geojson", # geojson file
    style_callback=make_style_callback("optimum_area_diff", cmap, norm), # style
    name="Optimum grid", # name for legend
)
line_optimum_layer = GeoJSONLayerManager(
    "stations2grid_optimum_line.geojson", # geojson file
    style_callback=lambda feature: vector_style(feature, "black", opacity=0.8, weight=3), # style
    name="Station to optimum grid", # name for legend
)
river_overlay = GeoJSONLayerManager(
    "riv_seq_europe.geojson", # geojson file
    style_callback=lambda feature: vector_style(feature, "blue", 0.1), # style
    name="River grid", # name for legend
)

#### 2. Add and visualise layers in the interactive map

In [2]:
# Create an InteractiveMap instance
my_map = InteractiveMap()

# Add layers to the map
my_map.add_layer(river_overlay)
my_map.add_layer(station_layer)
my_map.add_layer(nearest_grid_layer)
my_map.add_layer(optimum_grid_layer)
my_map.add_layer(line_optimum_layer)

# Add line click handlers after the layer has been added to the map
if line_optimum_layer.layer:
    line_optimum_layer.layer.on_click(
        make_line_click_handler(
            "station_name",
            "station_area",
            "near_grid_area",
            "optimum_grid_area",
            "optimum_distance_cells",
            my_map.map,  # Pass the map object as an argument
        )
    )

# Create the legend widget
legend_widget = create_gradient_legend(cmap, vmin, vmax)
my_map.map.add_control(WidgetControl(widget=legend_widget, position="bottomright"))

# Display the map using IPython's display function
display(my_map.map)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…